In [ ]:
import numpy as np
import pandas as pd
import keras
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.preprocessing import normalize
from keras.models import Model, Sequential
from keras.layers import Dense,BatchNormalization,Conv2D, Conv1D
from keras.layers import Input,GlobalMaxPooling2D,concatenate, GlobalMaxPooling1D, Flatten
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping,ReduceLROnPlateau,ModelCheckpoint
from os import listdir, getcwd, chdir
from os.path import isfile, join

In [ ]:
workbookDir = getcwd()
print('Current Notebook Dir: ' + workbookDir)
chdir(workbookDir) # Go to current workbook Dir"
chdir('..')        # Go up one
print(f'Using current directory for loading data: {getcwd()}')
workbookDir = getcwd()

In [ ]:
data_path = '../diff_predictor/raw_data_pnn/'
chabc_train_filelist = [f for f in listdir(data_path) if isfile(join(data_path, f)) and 'ChABC' in f and 'ChABC_brain_4' not in f]
chabc_validation_filelist = [f for f in listdir(data_path) if isfile(join(data_path, f)) and 'ChABC_brain_4' in f]
nt_train_filelist = [f for f in listdir(data_path) if isfile(join(data_path, f)) and 'NT' in f and 'NT_brain_4' not in f]
nt_validation_filelist = [f for f in listdir(data_path) if isfile(join(data_path, f)) and 'NT_brain_4' in f]


print(len(nt_validation_filelist)) #Check to make sure there are the right number of files, in this case 180 - 60 per class
print(len(chabc_validation_filelist))

In [ ]:
filelist[0]

In [ ]:
label = np.zeros([64,1])
label.shape

In [ ]:
out = np.zeros([64,100-1,1])
out.shape

In [ ]:
partition = {
    'train': df['Track_ID'].unique()
}

In [ ]:
df

In [ ]:
partition = dict()

In [ ]:
training_filelist = chabc_train_filelist + nt_train_filelist
print(len(training_filelist))

In [ ]:
#set up training files and labels

lengths = []

track_ids = []
training_labels = dict()
for file in training_filelist:

    df = pd.read_csv(data_path + file)

    for track_id in df['Track_ID'].unique():
        label = file[:-4] + f'_track_{int(track_id)}'
        if "NT" in file:
            training_labels[str(label)] = 1
        elif "ChABC" in file:
            training_labels[str(label)] = 0
        
        track_data = df[df['Track_ID']==track_id][['X', 'Y']]
        track_data = track_data.fillna(0)
    #     #dataset[str(track_id)] = np.array(track_data)

        test_array = np.array(track_data)
        #norm_array = normalize(test_array)
        # print(np.count_nonzero(test_array))
        # lengths.append(len(test_array))
        #print(np.count_nonzero(np.isnan(test_array)))
        if np.count_nonzero(test_array) >19:
            print(np.count_nonzero(test_array))
            track_ids.append(str(label))
            np.save(str(f'data/treatment_training_data/{file}').replace(".csv", f"_track_{int(track_id)}"), test_array)

partition['train'] = track_ids
   

In [ ]:
array_lengths = np.array(lengths)
print(len(array_lengths))
np.count_nonzero(array_lengths>50)

In [ ]:
validation_filelist = nt_validation_filelist + chabc_validation_filelist
print(len(validation_filelist))

In [ ]:
#set up validation files and labels

track_ids = []
validation_labels = dict()
for file in validation_filelist:

    df = pd.read_csv(data_path + file)

    for track_id in df['Track_ID'].unique():
        label = file[:-4] + f'_track_{int(track_id)}'
        if "NT" in file:
            validation_labels[str(label)] = 1
        else:
            validation_labels[str(label)] = 0
        
        track_data = df[df['Track_ID']==track_id][['X', 'Y']]
        track_data = track_data.fillna(0)
    #     #dataset[str(track_id)] = np.array(track_data)
        test_array = np.array(track_data)
        #norm_array = normalize(test_array)
        if np.count_nonzero(test_array) >19:
            track_ids.append(str(label))
            np.save(str(f'data/treatment_validation_data/{file}').replace(".csv", f"_track_{int(track_id)}"), test_array)

partition['validation'] = track_ids

In [ ]:
test_array[:,0]

In [ ]:
cnn_data_path = '../diff_predictor/data/10div_training_data/'
one_um_filelist = [f for f in listdir(cnn_data_path) if isfile(join(cnn_data_path, f)) and '1uM' in f]

nt_filelist = [f for f in listdir(cnn_data_path) if isfile(join(cnn_data_path, f)) and 'NT' in f]
len(one_um_filelist)

In [ ]:
out = out[out!=0]
out.shape

In [ ]:
counter = 0


In [ ]:
fig, axes = plt.subplots(5, 5, figsize=(10,10))
for i in range(5):
    for j in range(5):
        file = np.load(cnn_data_path+one_um_filelist[counter])
        if np.count_nonzero(file) >0:
            counter += 1
            axes[i,j].plot(file[:,0], file[:,1])
print(counter)

In [ ]:
fig, axes = plt.subplots(5, 5, figsize=(10,10))
for i in range(5):
    for j in range(5):
        file = np.load(cnn_data_path+nt_filelist[counter])
        counter += 1
        axes[i,j].plot(file[:,0], file[:,1])
print(counter)

In [ ]:
partition.keys

In [ ]:
dataset[track_ids[0]]

In [ ]:
class MptDataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, data_path, list_IDs, labels, batch_size=128, dim=(651,2), n_channels=None,
                 n_classes=2, shuffle=True):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.labels = labels
        self.list_IDs = list_IDs
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.data_path = data_path
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.empty((self.batch_size, *self.dim))#, self.n_channels))
        y = np.empty((self.batch_size), dtype=int)

        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            # Store sample
            out_array = np.load(self.data_path + ID + '.npy')
            out_array = normalize(out_array)
            X[i,] = out_array

            # Store class
            y[i] = self.labels[ID]

        return X, keras.utils.to_categorical(y, num_classes=self.n_classes)


In [ ]:
training_labels

In [ ]:
training_generator = MptDataGenerator('../diff_predictor/data/treatment_training_data/', partition['train'], training_labels)

In [ ]:
validation_generator = MptDataGenerator('../diff_predictor/data/treatment_validation_data/', partition['validation'], validation_labels)

In [ ]:
inputs = Input((651, 2))
initializer = 'random_normal'
f = 16

In [ ]:
x1 = Conv1D(f,4,padding='causal',activation='relu',kernel_initializer=initializer)(inputs)
x1 = BatchNormalization()(x1)
x1 = Conv1D(f,4,dilation_rate=2,padding='causal',activation='relu',kernel_initializer=initializer)(x1)
x1 = BatchNormalization()(x1)
x1 = Conv1D(f,4,dilation_rate=4,padding='causal',activation='relu',kernel_initializer=initializer)(x1)
x1 = BatchNormalization()(x1)
x1 = GlobalMaxPooling1D()(x1)

dense = Dense(512,activation='relu')(x1)
dense = Dense(256,activation='relu')(dense)
dense2 = Dense(2,activation='softmax')(dense)
model = Model(inputs=inputs, outputs=dense2)

In [ ]:
x1 = Conv1D(f,4,padding='causal',activation='relu',kernel_initializer=initializer)(inputs)
x1 = BatchNormalization()(x1)
x1 = Conv1D(f,4,dilation_rate=2,padding='causal',activation='relu',kernel_initializer=initializer)(x1)
x1 = BatchNormalization()(x1)
x1 = Conv1D(f,4,dilation_rate=4,padding='causal',activation='relu',kernel_initializer=initializer)(x1)
x1 = BatchNormalization()(x1)
x1 = GlobalMaxPooling1D()(x1)


x2 = Conv1D(f,2,padding='causal',activation='relu',kernel_initializer=initializer)(inputs)
x2 = BatchNormalization()(x2)
x2 = Conv1D(f,2,dilation_rate=2,padding='causal',activation='relu',kernel_initializer=initializer)(x2)
x2 = BatchNormalization()(x2)
x2 = Conv1D(f,2,dilation_rate=4,padding='causal',activation='relu',kernel_initializer=initializer)(x2)
x2 = BatchNormalization()(x2)
x2 = GlobalMaxPooling1D()(x2)


x3 = Conv1D(f,3,padding='causal',activation='relu',kernel_initializer=initializer)(inputs)
x3 = BatchNormalization()(x3)
x3 = Conv1D(f,3,dilation_rate=2,padding='causal',activation='relu',kernel_initializer=initializer)(x3)
x3 = BatchNormalization()(x3)
x3 = Conv1D(f,3,dilation_rate=4,padding='causal',activation='relu',kernel_initializer=initializer)(x3)
x3 = BatchNormalization()(x3)
x3 = GlobalMaxPooling1D()(x3)


x4 = Conv1D(f,10,padding='causal',activation='relu',kernel_initializer=initializer)(inputs)
x4 = BatchNormalization()(x4)
x4 = Conv1D(f,10,dilation_rate=4,padding='causal',activation='relu',kernel_initializer=initializer)(x4)
x4 = BatchNormalization()(x4)
x4 = Conv1D(f,10,dilation_rate=8,padding='causal',activation='relu',kernel_initializer=initializer)(x4)
x4 = BatchNormalization()(x4)
x4 = GlobalMaxPooling1D()(x4)


x5 = Conv1D(f,20,padding='same',activation='relu',kernel_initializer=initializer)(inputs)
x5 = BatchNormalization()(x5)
x5 = GlobalMaxPooling1D()(x5)


con = concatenate([x1,x2,x3,x4,x5])
dense = Dense(512,activation='relu')(con)
dense = Dense(128,activation='relu')(dense)
dense2 = Dense(2,activation='softmax')(dense)
model = Model(inputs=inputs, outputs=dense2)

optimizer = Adam(lr=1e-5)
model.compile(optimizer=optimizer,loss='categorical_crossentropy',metrics=['acc'])
model.summary()

In [ ]:
model = Sequential()
model.add(Flatten())
model.add(Dense(1000, input_dim=inputs, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(2, activation='softmax'))

# con = concatenate([x1,x2,x3,x4])
# dense = Dense(512,activation='relu')(con)
# dense = Dense(256#,activation='relu')(dense)
# dense2 = Dense(1,activation='sigmoid')(dense)
# model = Model(inputs=inputs, outputs=dense2)

# optimizer = Adam(lr=1e-5)
# model.compile(optimizer=optimizer,loss='mse',metrics=['mse'])


In [ ]:
model.compile(loss='mse',
              optimizer=Adam(lr=1e-5),
              metrics=['mse', 'accuracy'])

model.fit(training_generator, epochs=3, validation_data=validation_generator)

In [ ]:
results = model.predict(validation_generator, batch_size=32)

In [ ]:
results

In [ ]:
from sklearn.preprocessing import scale
output = scale()
output

In [ ]:
x_y = np.array(df[df['Track_ID']== 1][['X', 'Y']])
x_y

In [ ]:
norm = np.linalg.norm(x_y, 1)
im = x_y/norm
plt.figure(figsize=(10,500))
plt.imshow(im)

In [ ]:
x_y[:2,:]

In [ ]:
def scale_array(A):
    return (A-np.min(A))/(np.max(A) - np.min(A))

output = scale_array(x_y[:19,:])
output

In [ ]:
import cv2
%matplotlib inline
im = np.array(output).astype('uint8')
plt.figure(figsize=(10,10))
plt.imshow(im, cmap='Greys')
# grayImage = cv2.cvtColor(uint_img, cv2.COLOR_GRAY2BGR)
# cv2.imshow(mat=grayImage)

In [ ]:
plt.plot(x_y)

In [ ]:
def traj_generator(batch_size, datapath):
    out = np.zeros([batch_size, 651, 2]) #initialize trajectory array
    label = np.zeros([batch_size, 1]) #initialize label array
    batch_count = 0
    while True:
        df = pd.read_csv(datapath)
        for i in range(batch_size):
            batch_count += 1
            x_y = np.array(df[df['Track_ID']==i][['X', 'Y']])
            out[i,:,:] = x_y
            label[i] = 1
        yield out, label


In [ ]:
inputs = Input((651, 2))
initializer = 'random_normal'
f = 32

In [ ]:
x1 = Conv1D(f,4,padding='causal',activation='relu',kernel_initializer=initializer)(inputs)
x1 = BatchNormalization()(x1)
x1 = Conv1D(f,4,dilation_rate=2,padding='causal',activation='relu',kernel_initializer=initializer)(x1)
x1 = BatchNormalization()(x1)
x1 = Conv1D(f,4,dilation_rate=4,padding='causal',activation='relu',kernel_initializer=initializer)(x1)
x1 = BatchNormalization()(x1)
x1 = GlobalMaxPooling1D()(x1)


x2 = Conv1D(f,2,padding='causal',activation='relu',kernel_initializer=initializer)(inputs)
x2 = BatchNormalization()(x2)
x2 = Conv1D(f,2,dilation_rate=2,padding='causal',activation='relu',kernel_initializer=initializer)(x2)
x2 = BatchNormalization()(x2)
x2 = Conv1D(f,2,dilation_rate=4,padding='causal',activation='relu',kernel_initializer=initializer)(x2)
x2 = BatchNormalization()(x2)
x2 = GlobalMaxPooling1D()(x2)


x3 = Conv1D(f,3,padding='causal',activation='relu',kernel_initializer=initializer)(inputs)
x3 = BatchNormalization()(x3)
x3 = Conv1D(f,3,dilation_rate=2,padding='causal',activation='relu',kernel_initializer=initializer)(x3)
x3 = BatchNormalization()(x3)
x3 = Conv1D(f,3,dilation_rate=4,padding='causal',activation='relu',kernel_initializer=initializer)(x3)
x3 = BatchNormalization()(x3)
x3 = GlobalMaxPooling1D()(x3)


x4 = Conv1D(f,10,padding='causal',activation='relu',kernel_initializer=initializer)(inputs)
x4 = BatchNormalization()(x4)
x4 = Conv1D(f,10,dilation_rate=5,padding='causal',activation='relu',kernel_initializer=initializer)(x4)
x4 = BatchNormalization()(x4)
x4 = Conv1D(f,10,dilation_rate=10,padding='causal',activation='relu',kernel_initializer=initializer)(x4)
x4 = BatchNormalization()(x4)
x4 = GlobalMaxPooling1D()(x4)

con = concatenate([x1,x2,x3,x4])
dense = Dense(512,activation='relu')(con)
dense = Dense(256,activation='relu')(dense)
dense2 = Dense(1,activation='sigmoid')(dense)
model = Model(inputs=inputs, outputs=dense2)

optimizer = Adam(lr=1e-5)
model.compile(optimizer=optimizer,loss='mse',metrics=['mse', 'accuracy'])
model.summary()

In [ ]:
model = Sequential()
model.add(Dense(16, input_dim=1, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='linear'))

In [ ]:
model.compile(loss='mean_absolute_error',
              optimizer=Adam(),
              metrics=['mean_squared_error'])

In [ ]:
model.compile(loss='mean_absolute_error',
              optimizer=Adam(),
              metrics=['mean_squared_error', 'accuracy'])

model.fit(training_generator, epochs=2, validation_data=validation_generator)